# How to deploy ADK Apps to Agent Engine

[Documentation link 🔗](https://google.github.io/adk-docs/deploy/agent-engine/)

## 1. Set the local environment variables
These will be used in the Agent Engine deployment

In [4]:
from dotenv import load_dotenv
import os

load_dotenv("../trends_and_insights_agent/.env")  # take environment variables

True

## 2. Set up the `AdkApp` Convenience wrapper for ADK Agents -> Agent Engine

Note the injection of the environment variables.

In [ ]:
from vertexai.preview.reasoning_engines import AdkApp

from trends_and_insights_agent import agent

env_vars = {}

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["BUCKET"] = os.getenv("BUCKET")
env_vars["GOOGLE_CLOUD_PROJECT_NUMBER"] = os.getenv("GOOGLE_CLOUD_PROJECT_NUMBER")
env_vars["YT_SECRET_MNGR_NAME"] = os.getenv("YT_SECRET_MNGR_NAME")


my_agent = AdkApp(agent=agent.root_agent, enable_tracing=False, env_vars=env_vars)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:root:

`SESSION_STATE_JSON_PATH`: None


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


# 3. Test the Agent Engine locally

In [6]:
session = my_agent.create_session(user_id="jwortz")
session

Session(id='6f0ac3fd-9f16-4ede-bfe7-43aadc896d82', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1751991656.119463)

In [7]:
my_agent.list_sessions(user_id="jwortz")

ListSessionsResponse(sessions=[Session(id='6f0ac3fd-9f16-4ede-bfe7-43aadc896d82', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1751991656.119463)])

In [8]:
EXAMPLE_BRIEF = """
Campaign Name: Connect Beyond Limits with Pix Phone 5
Campaign Objectives
● Increase awareness of the latest model of the Pix Phone
● Generate leads and drive phone sales
● Position Pix Phone and the trendy phone to have
Target Audience
● Individuals aged 20-40 in major markets such as US, France, Japan
Media Strategy
1. Social Media Marketing: Run targeted social media ads on platforms where the
target audience is active.
2. Influencer Marketing: Partner with influencers in the tech industry to promote Pix
Phone 5
3. Paid Advertising: Run targeted display ads on websites and apps frequented by
the target audience. Use search engine marketing (SEM) to bid on relevant
keywords and appear in search results when potential customers are looking for
asset protection insurance.
Timeline and country
● Activity in the 3 major markets in at least the online channels by early Oct 2023
● Start from US, followed by France then Japan
● The campaign should use the US version as anchor and localized accordingly
Performance Metrics
1. Track website conversion, lead generation, and phone sales to measure campaign
effectiveness.
2. Use social media analytics to monitor engagement, reach, and sentiment.
"""

In [9]:
from pprint import pprint


def stream_agent(agent, prompt: str, session) -> None:
    try:
        session_id = session['id']
    # local prefers attributes vs. remote prefers dict keys
    except TypeError:
        session_id = session.id
    except Exception as e:
        return f"Session Object not valid: {e}"
        
    for event in agent.stream_query(
        user_id="jwortz",
        session_id=session_id,
        message=prompt,
    ):
        pprint(event)

In [10]:
first_prompt = f"Check out the {EXAMPLE_BRIEF} and come up with some good campaign ideas. Be sure to generate a few image ideas for the campaign."
stream_agent(my_agent, first_prompt, session)

INFO:root:

Loading Initial State (empty): {'state': {'artifact_keys': {'image_creatives': {}, 'video_creatives': {}}, 'target_product': '', 'target_audience': [], 'key_selling_points': [], 'campaign_guide': {'campaign_name': '', 'brand': '', 'target_product': '', 'target_region': '', 'target_audience': [], 'campaign_objectives': [], 'media_strategy': {}, 'key_selling_points': []}, 'target_search_trends': {'target_search_trends': []}, 'target_yt_trends': {'target_yt_trends': []}}}


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:

You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.
Do not perform any research yourself. Your job is to **Delegate**.


You are an Expert AI Marketing Resear

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'artifact_keys': {'image_creatives': {},
                                               'video_creatives': {}},
                             'campaign_guide': {'brand': '',
                                                'campaign_name': '',
                                                'campaign_objectives': [],
                                                'key_selling_points': [],
                                                'media_strategy': {},
                                                'target_audience': [],
                                                'target_product': '',
                                                'target_region': ''},
                             'key_selling_points': [],
                             'target_audience': [],
                             'target_product': '',
             

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
None
-----------------------------------------------------------
Function calls:
name: transfer_to_agent, args: {'agent_name': 'campaign_guide_data_generation_agent'}
-----------------------------------------------------------
Raw response:
{"candidates":[{"content":{"parts":[{"function_call":{"args":{"agent_name":"campaign_guide_data_generation_agent"},"name":"transfer_to_agent"}}],"role":"model"},"finish_reason":"STOP","avg_logprobs":-0.601440149195054}],"create_time":"2025-07-08T16:21:02.200484Z","response_id":"bkVtaKSeDJyimecPl-GQgQw","model_version":"gemini-2.5-flash","usage_metadata":{"candidates_token_count":17,"candidates_tokens_details":[{"modality":"TEXT","token_count":17}],"prompt_token_count":1274,"prompt_tokens_details":[{"modality":"TEXT","token_count":1274}],"thoughts_token_count":73,"total_token_count":1364,"traffic_type":"ON_DEMAND"},"automatic_

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'root_agent',
 'content': {'parts': [{'function_call': {'args': {'agent_name': 'campaign_guide_data_generation_agent'},
                                          'id': 'adk-be01330e-d734-413f-9772-165ef0db6e01',
                                          'name': 'transfer_to_agent'}}],
             'role': 'model'},
 'id': '7bZuKU9v',
 'invocation_id': 'e-0a3047a6-9d34-4000-8446-3c81e14ce7d8',
 'long_running_tool_ids': set(),
 'timestamp': 1751991661.203292,
 'usage_metadata': {'candidates_token_count': 17,
                    'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                                   'token_count': 17}],
                    'prompt_token_count': 1274,
                    'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                               'token_count': 12

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
None
-----------------------------------------------------------
Function calls:
name: transfer_to_agent, args: {'agent_name': 'ad_content_generator_agent'}
-----------------------------------------------------------
Raw response:
{"candidates":[{"content":{"parts":[{"function_call":{"args":{"agent_name":"ad_content_generator_agent"},"name":"transfer_to_agent"}}],"role":"model"},"finish_reason":"STOP","avg_logprobs":-1.7483516693115235}],"create_time":"2025-07-08T16:21:03.905318Z","response_id":"b0VtaOagN5qBm9IP_saF0AY","model_version":"gemini-2.5-flash","usage_metadata":{"candidates_token_count":15,"candidates_tokens_details":[{"modality":"TEXT","token_count":15}],"prompt_token_count":861,"prompt_tokens_details":[{"modality":"TEXT","token_count":861}],"thoughts_token_count":312,"total_token_count":1188,"traffic_type":"ON_DEMAND"},"automatic_function_calling_his

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'campaign_guide_data_generation_agent',
 'content': {'parts': [{'function_call': {'args': {'agent_name': 'ad_content_generator_agent'},
                                          'id': 'adk-cb954c14-e337-4bea-ab52-99cd7c2a3e66',
                                          'name': 'transfer_to_agent'}}],
             'role': 'model'},
 'id': 'KBunPwNi',
 'invocation_id': 'e-0a3047a6-9d34-4000-8446-3c81e14ce7d8',
 'long_running_tool_ids': set(),
 'timestamp': 1751991663.115112,
 'usage_metadata': {'candidates_token_count': 15,
                    'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                                   'token_count': 15}],
                    'prompt_token_count': 861,
                    'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                               't

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
None
-----------------------------------------------------------
Function calls:
name: transfer_to_agent, args: {'agent_name': 'ad_creative_pipeline'}
-----------------------------------------------------------
Raw response:
{"candidates":[{"content":{"parts":[{"function_call":{"args":{"agent_name":"ad_creative_pipeline"},"name":"transfer_to_agent"}}],"role":"model"},"finish_reason":"STOP","avg_logprobs":-1.9861051119290865}],"create_time":"2025-07-08T16:21:07.286467Z","response_id":"c0VtaIO-EdmJm9IPq8nk4Ak","model_version":"gemini-2.5-pro","usage_metadata":{"candidates_token_count":13,"candidates_tokens_details":[{"modality":"TEXT","token_count":13}],"prompt_token_count":1112,"prompt_tokens_details":[{"modality":"TEXT","token_count":1112}],"thoughts_token_count":269,"total_token_count":1394,"traffic_type":"ON_DEMAND"},"automatic_function_calling_history":[]}
--

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'ad_content_generator_agent',
 'content': {'parts': [{'function_call': {'args': {'agent_name': 'ad_creative_pipeline'},
                                          'id': 'adk-a4652f4f-2f27-4ecb-89f3-3df0b2d8a715',
                                          'name': 'transfer_to_agent'}}],
             'role': 'model'},
 'id': 'cAsV8D9O',
 'invocation_id': 'e-0a3047a6-9d34-4000-8446-3c81e14ce7d8',
 'long_running_tool_ids': set(),
 'timestamp': 1751991666.470908,
 'usage_metadata': {'candidates_token_count': 13,
                    'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                                   'token_count': 13}],
                    'prompt_token_count': 1112,
                    'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                               'token_count': 11

In [11]:
followup_prompt = "Create images from these great ideas!"
stream_agent(my_agent, followup_prompt, session)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-pro, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:

You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.
Do not perform any research yourself. Your job is to **Delegate**.


**Role:** You are the orchestrator for a comprehensive ad content generation workflow.

**Objective:** Coordinate two specialized subagents to create a complete set of ad creatives including copy, images, and videos.

**Workflow:**
1. First, transfer to the `ad_creative_pipeline` to generate 4-8 ad copy options.
2. Once ad copies are selected, transfer to the `visual_generation_pipeline` to create 4-8 visual options for each selected copy.
3. Once the prompts are created and selected, use the `visual_generator` to generat

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'ad_content_generator_agent',
 'content': {'parts': [{'function_call': {'args': {'agent_name': 'visual_generation_pipeline'},
                                          'id': 'adk-dc115094-e783-4e35-bd03-12f17f5dd2d6',
                                          'name': 'transfer_to_agent'}}],
             'role': 'model'},
 'id': 'wKzJkO12',
 'invocation_id': 'e-d7a9e626-321f-47b1-bc37-2f0f58138c42',
 'long_running_tool_ids': set(),
 'timestamp': 1751991670.714077,
 'usage_metadata': {'candidates_token_count': 13,
                    'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                                   'token_count': 13}],
                    'prompt_token_count': 1139,
                    'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                               'token_coun

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Here are several visual concepts for the "Connect Beyond Limits with Pix Phone 5" campaign, designed to resonate with a trendy, tech-savvy audience in the US, France, and Japan. The concepts incorporate current visual trends to ensure the campaign feels fresh and engaging.

### **Ad Copy Themes (Assumed)**

*   **Ad Copy 1 (Seamless Ecosystem):** "Pix Phone 5: Your world, one seamless flow. From your morning routine to your evening unwind, stay connected without missing a beat."
*   **Ad Copy 2 (AI-Powered Camera):** "Pix Phone 5: Capture reality, then perfect it. AI-powered photography that makes every shot your best."
*   **Ad Copy 3 (Limitless Creativity):** "Pix Phone 5: Your creative studio on the go. Shoot, edit, and share your vision with the world, faster than ever."
*   **Ad Copy 4 (Global Connection):** "Pix Phone 5 with Live Translate: Speak the world

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'visual_draft': '**Grounding My Search**\n'
                                             '\n'
                                             '\\n\\n\n'
                                             '\n'
                                             "I've completed the initial "
                                             'search queries to establish '
                                             'current visual trends. Based on '
                                             'the search results, I am seeing '
                                             'a prevalence of '
                                             'authentic-feeling content, use '
                                             'of UGC, seamless transitions, '
                                             "and minimalist aesthetics. I'll "
                                             'move on to crafting visual '
      

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
After a thorough review of the four visual concepts for the "Connect Beyond Limits with Pix Phone 5" campaign, I am pleased to approve all four for progression. Each concept is strong, strategically sound, and leverages distinct, relevant trends that will resonate with the target audience across the US, France, and Japan.

The concepts present a diverse and compelling portfolio, balancing video and static images, and ranging in tone from sleekly modern to authentically human. The prompts are exceptionally well-crafted, providing the verbose, detailed direction needed for high-quality AI generation while maintaining continuity and clarity.

Here is a detailed rationale for each selection:

---

### **Selection 1: The Seamless Flow (Video)**

This concept is a standout for its clear, modern, and highly relatable storytelling. It directly visualizes the core benefi

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'selected_concepts': '**Forming Selections**\n'
                                                  '\n'
                                                  "I've completed my initial "
                                                  'analysis of all four '
                                                  'concepts, considering their '
                                                  'visual appeal, alignment '
                                                  'with the ad copy, trend '
                                                  'relevance, and feasibility. '
                                                  'Now, I plan to execute the '
                                                  'search queries I identified '
                                                  'earlier to validate and '
                                                  'support my assessments. '
     

## 4. Deploy to Vertex AI Agent Engine
To call and manage agents in production, deploy the agent to Vertex AI Agent Engine.

**Important - run `poetry build` to package the agent**

In [12]:
! poetry build --format=wheel --output=notebooks/deployment

Building trends_and_insights_agent (0.1.0)
  - Building wheel
  - Built trends_and_insights_agent-0.1.0-py3-none-any.whl


#### Initialize the Vertex client, then create a `remote_agent` that is deployed to Vertex

This also takes the packaged agent code. This is required for more complex agents that have nested dependencies and require packaging

Also, before running - be sure to give Secret Manager access to the Agent Engine service account. This can simply be done by running `. setup_ae_sm_access.sh`.

The code can also be ran as follows (relative to repo root):

```bash
source trends_and_insights_agent/.env

export RE_SA="service-${GOOGLE_CLOUD_PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
gcloud secrets add-iam-policy-binding "projects/$GOOGLE_CLOUD_PROJECT/secrets/$YT_SECRET_MNGR_NAME" \
  --member="serviceAccount:$RE_SA" \
  --role="roles/secretmanager.secretAccessor"
```

In [ ]:
import vertexai
from vertexai import agent_engines

GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
APP_NAME = "retail-data-agent"
BUCKET = os.getenv("BUCKET")

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location="us-central1",
    staging_bucket=BUCKET,
)

remote_agent = agent_engines.create(
    agent_engine=my_agent,
    display_name="trends-and-insights",
    description="You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.",
    requirements=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
    extra_packages=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
    env_vars=env_vars,
)

Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.97.0'}


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.97.0'}


Failed to parse constraint: deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl. Exception: Expected end or semicolon (after name and no valid version specifier)
    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


The following requirements are missing: {'pydantic', 'cloudpickle', 'google-cloud-aiplatform'}


...

INFO:vertexai.agent_engines:Creating AgentEngine


Create AgentEngine backing LRO: projects/679926387543/locations/us-central1/reasoningEngines/9102364185148260352/operations/5538097373114269696


INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/679926387543/locations/us-central1/reasoningEngines/9102364185148260352/operations/5538097373114269696


View progress and logs at https://console.cloud.google.com/logs/query?project=wortz-project-352116


INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=wortz-project-352116


### Save the agent engine resource name

This can be used by the command to deploy it to Agentspace

```bash
agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1093257605637210112')
```

## 5. Try it remotely

In [3]:
session1 = remote_agent.create_session(user_id="jwortz")
session1

NameError: name 'remote_agent' is not defined

#### Same idea above applies to remote agents

In [26]:
first_prompt = f"Check out the {EXAMPLE_BRIEF} and come up with some good campaign ideas. Be sure to generate a few image ideas for the campaign."
stream_agent(remote_agent, first_prompt, session1)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'marketing_idea_generator_agent',
 'content': {'parts': [{'text': "Hello! I'm here to help you brainstorm some "
                                'great campaign ideas for the "Connect Beyond '
                                'Limits with Pix Phone 5" campaign.\n'
                                '\n'
                                'To start, could you please provide the '
                                'marketing campaign guide? You can paste the '
                                'text, provide a URL to a webpage, or upload a '
                                'PDF. This will help me get a better '
                                'understanding of the campaign and your '
                                'goals.\n'
                                '\n'
                                'For example, you could say:\n'
                                '\n'
                 

# 6. Optional Cleanup

In [ ]:
remote_agent.delete(force=True)